Step 1-Cleaning the Raw document and preprocess the text(remove noise and normalize encoding) , Best approach is to remove extra whitespace using python's built in regular expression library(re)

In [ ]:
import re
import textwrap

# 1. Document Creation / Ingestion
# We'll use the "Dataset" generated aritfically using an LLM which is suitable for graphrag.
# It is stored here as a multiline string.

doc_text = """
Whitepaper: AuraFlow - A Decentralized Framework for Adaptive AI

1. Introduction

In the landscape of artificial intelligence, traditional monolithic systems present significant challenges in scalability, adaptability, and resilience. AuraFlow is a novel, decentralized framework designed to overcome these limitations by enabling the creation of stateful, multi-agent AI systems. It is built on the core principles of decentralization, modularity, and emergent intelligence. By distributing tasks across specialized, independent agents, AuraFlow creates robust systems that can adapt to new information in real-time without requiring complete model retraining. This document outlines the core components, architecture, and primary use cases of the AuraFlow framework.

---

2. Core Components

The AuraFlow framework is composed of four primary components that work in synergy. Each component is a specialized agent with a distinct role.

2.1 The Cognition Core
The Cognition Core is the central reasoning engine of an AuraFlow instance. Unlike traditional neural networks, it utilizes a proprietary Probabilistic Logic Network (PLN) for decision-making. This allows the Core to handle uncertainty, reason with incomplete information, and provide transparent, explainable outputs. The Cognition Core is responsible for high-level task decomposition, planning, and final response synthesis. Its performance is heavily dependent on the quality of processed data it receives from the Data Weavers.

2.2 Data Weavers
Data Weavers are specialized agents tasked with data ingestion, preprocessing, and normalization. Each Weaver can be configured to handle specific data modalities, such as unstructured text, images, streaming time-series data, or structured database records. They clean and transform raw data into a standardized format that the Cognition Core can efficiently process. This modular approach allows an AuraFlow system to seamlessly integrate new data sources by simply deploying a new, appropriately configured Data Weaver. Communication between Data Weavers and the Cognition Core is managed by the Synapse Bridge.

2.3 The Synapse Bridge
The Synapse Bridge is the high-bandwidth, low-latency communication backbone of the AuraFlow framework. It facilitates interaction between all components, primarily managing the flow of information from the Data Weavers to the Cognition Core and broadcasting the Core's directives to other agents. It uses a custom, lightweight data exchange protocol called the Neuro-Link Protocol, which ensures secure and efficient data transmission, a critical feature for real-time applications.

2.4 The Sentinel Layer
The Sentinel Layer acts as the ethical and security guardian of the system. It is a specialized validation agent that monitors the outputs and behavior of the Cognition Core in real-time. The Sentinel Layer is responsible for applying ethical constraints, enforcing operational boundaries, and preventing the generation of harmful or biased outputs. It can veto or flag a decision made by the Cognition Core if it violates pre-defined rules, ensuring that the system operates safely and responsibly.

---

3. System Architecture and Data Flow

The architecture of AuraFlow is inherently decentralized. A typical workflow for processing a user query follows these steps:
1.  A query is received by the system.
2.  Relevant Data Weavers are activated to gather and process external or internal data related to the query.
3.  The processed data is transmitted securely via the Synapse Bridge to the Cognition Core.
4.  The Cognition Core uses its PLN to analyze the data, reason about the query, and formulate a plan or response.
5.  Before being finalized, the proposed response is sent to the Sentinel Layer for validation.
6.  If approved, the final response is generated and delivered.

This modular data flow ensures that each component can be independently upgraded or scaled. For instance, if processing speed becomes a bottleneck, more Data Weaver instances can be deployed without altering the Cognition Core.

---

4. Key Applications

The unique architecture of AuraFlow makes it suitable for complex, dynamic environments.

Real-time Market Analysis: An AuraFlow system can deploy multiple Data Weavers to monitor financial news, social media sentiment, and stock market data simultaneously. The Cognition Core can then synthesize this information to identify trends and risks, while the Sentinel Layer ensures that trading recommendations adhere to regulatory compliance.

Autonomous Scientific Research: In this scenario, a network of AuraFlow instances can collaborate on research. One instance could use its Data Weavers to analyze experimental data from lab equipment, while another analyzes existing scientific literature. The Cognition Cores could then exchange findings via their Synapse Bridges to formulate new hypotheses, accelerating the pace of discovery.
"""

# 2. Clean and preprocess text
def preprocess_text(text: str) -> str:
    """A simple function to clean up text data."""
    # Replace multiple newlines with a single one
    text = re.sub(r'\n+', '\n', text)
    # Replace multiple spaces with a single space
    text = re.sub(r' +', ' ', text)
    # Strip leading/trailing whitespace
    text = text.strip()
    return text

# Apply the preprocessing function to our document
cleaned_doc = preprocess_text(doc_text)

# --- Verification (Optional) ---
# Print the first 500 characters of the cleaned document to see the result.
print("--- Cleaned Document (First 500 Characters) ---")
# textwrap.fill helps in pretty-printing the text to fit the screen width
print(textwrap.fill(cleaned_doc[:500], width=80))

Step 2-Chunking and Vectorization , We implement the chunking process using the spaCy NLP library which breaks the document into individual sentence , Then for vectorization we will use a sentence transformer

In [ ]:
# Step 1: Install all required libraries
!pip install -q spacy sentence-transformers

# Step 2: Download the spaCy English model
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import re
import numpy as np
from sentence_transformers import SentenceTransformer

# The cleaned document from our first step.
cleaned_doc = """
Whitepaper: AuraFlow - A Decentralized Framework for Adaptive AI
1. Introduction
In the landscape of artificial intelligence, traditional monolithic systems present significant challenges in scalability, adaptability, and resilience. AuraFlow is a novel, decentralized framework designed to overcome these limitations by enabling the creation of stateful, multi-agent AI systems. It is built on the core principles of decentralization, modularity, and emergent intelligence. By distributing tasks across specialized, independent agents, AuraFlow creates robust systems that can adapt to new information in real-time without requiring complete model retraining. This document outlines the core components, architecture, and primary use cases of the AuraFlow framework.
2. Core Components
The AuraFlow framework is composed of four primary components that work in synergy. Each component is a specialized agent with a distinct role.
2.1 The Cognition Core
The Cognition Core is the central reasoning engine of an AuraFlow instance. Unlike traditional neural networks, it utilizes a proprietary Probabilistic Logic Network (PLN) for decision-making. This allows the Core to handle uncertainty, reason with incomplete information, and provide transparent, explainable outputs. The Cognition Core is responsible for high-level task decomposition, planning, and final response synthesis. Its performance is heavily dependent on the quality of processed data it receives from the Data Weavers.
2.2 Data Weavers
Data Weavers are specialized agents tasked with data ingestion, preprocessing, and normalization. Each Weaver can be configured to handle specific data modalities, such as unstructured text, images, streaming time-series data, or structured database records. They clean and transform raw data into a standardized format that the Cognition Core can efficiently process. This modular approach allows an AuraFlow system to seamlessly integrate new data sources by simply deploying a new, appropriately configured Data Weaver. Communication between Data Weavers and the Cognition Core is managed by the Synapse Bridge.
2.3 The Synapse Bridge
The Synapse Bridge is the high-bandwidth, low-latency communication backbone of the AuraFlow framework. It facilitates interaction between all components, primarily managing the flow of information from the Data Weavers to the Cognition Core and broadcasting the Core's directives to other agents. It uses a custom, lightweight data exchange protocol called the Neuro-Link Protocol, which ensures secure and efficient data transmission, a critical feature for real-time applications.
2.4 The Sentinel Layer
The Sentinel Layer acts as the ethical and security guardian of the system. It is a specialized validation agent that monitors the outputs and behavior of the Cognition Core in real-time. The Sentinel Layer is responsible for applying ethical constraints, enforcing operational boundaries, and preventing the generation of harmful or biased outputs. It can veto or flag a decision made by the Cognition Core if it violates pre-defined rules, ensuring that the system operates safely and responsibly.
3. System Architecture and Data Flow
The architecture of AuraFlow is inherently decentralized. A typical workflow for processing a user query follows these steps:
1. A query is received by the system.
2. Relevant Data Weavers are activated to gather and process external or internal data related to the query.
3. The processed data is transmitted securely via the Synapse Bridge to the Cognition Core.
4. The Cognition Core uses its PLN to analyze the data, reason about the query, and formulate a plan or response.
5. Before being finalized, the proposed response is sent to the Sentinel Layer for validation.
6. If approved, the final response is generated and delivered.
This modular data flow ensures that each component can be independently upgraded or scaled. For instance, if processing speed becomes a bottleneck, more Data Weaver instances can be deployed without altering the Cognition Core.
4. Key Applications
The unique architecture of AuraFlow makes it suitable for complex, dynamic environments.
Real-time Market Analysis: An AuraFlow system can deploy multiple Data Weavers to monitor financial news, social media sentiment, and stock market data simultaneously. The Cognition Core can then synthesize this information to identify trends and risks, while the Sentinel Layer ensures that trading recommendations adhere to regulatory compliance.
Autonomous Scientific Research: In this scenario, a network of AuraFlow instances can collaborate on research. One instance could use its Data Weavers to analyze experimental data from lab equipment, while another analyzes existing scientific literature. The Cognition Cores could then exchange findings via their Synapse Bridges to formulate new hypotheses, accelerating the pace of discovery.
"""

# --- 1. Chunking with spaCy ---
print("---  Loading spaCy model and chunking document... ---")
nlp = spacy.load("en_core_web_sm")
doc = nlp(cleaned_doc)
sentences = [sent.text.strip() for sent in doc.sents]

def create_sentence_chunks(sentences, chunk_size=500, overlap=50):
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk)
            overlap_text = ' '.join(current_chunk.split()[-overlap:])
            current_chunk = overlap_text + " " + sentence
        else:
            current_chunk += " " + sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

chunks = create_sentence_chunks(sentences, chunk_size=500, overlap=30)
print(f"---  Document chunked into {len(chunks)} parts. ---")
print("\nExample Chunk (Chunk 0):")
print(chunks[0])
print("-" * 20)


# --- 2. Vectorization (Embedding) ---
print("\n---  Loading embedding model and generating embeddings... ---")
# The first time this line runs, it will download the model. Please be patient.
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
embeddings = model.encode(chunks)

print(f"---  Embeddings Generated ---")
print(f"Shape of embeddings matrix: {embeddings.shape}")

3)Now storing the Created Embeddings in a database-chroma DB

In [ ]:
# Install the library for ChromaDB
!pip install -q chromadb

In [ ]:
import chromadb

# --- 1. Initialize ChromaDB Client ---
client = chromadb.Client()

# --- 2. Create a Collection ---
# A collection is where you'll store your embeddings, documents, and metadata.
collection = client.get_or_create_collection(name="auraflow_docs")

# --- 3. Prepare Data and Add to Collection ---
# ChromaDB requires a unique ID for each entry. We can simply use the index
# of each chunk as a string.
ids = [f"chunk_{i}" for i in range(len(chunks))]

# Add the data to the collection.
# This single command uploads your chunks, their embeddings, and their IDs.
collection.add(
    embeddings=embeddings,
    documents=chunks,
    ids=ids
)

# --- Verification (Optional) ---
# Check how many items are in the collection.
count = collection.count()
print(f"---  Collection '{collection.name}' created successfully. ---")
print(f"---  It contains {count} documents. ---")

# You can also peek at the first few items to see what they look like.
print("\n---  Peek at the first 2 items in the collection: ---")
peek_result = collection.peek(limit=2)
print(peek_result)

Step 4- Building the graph structure for Graph Rag-

Plan-
1) Each text chunk we created will become a node in our graph
2)Edges are relationships- We will connect these nodes based on two simple rules-
   a)Sequential Links-We will connect each chunk to chunk that comes immediately after it in the document
   b)Similarity links-for a given chunk we ffind other chunk that are sementically very similar we will connect them

In [ ]:
# Install scikit-learn for similarity calculations
!pip install -q scikit-learn

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. Initialize the Graph ---
# We'll use a dictionary where keys are chunk indices (nodes) and
# values are lists of connected chunk indices (edges).
graph = {i: [] for i in range(len(chunks))}


# --- 2. Add Sequential Edges ---
#  Connect each chunk to the one that follows it.
print("---  Adding sequential edges... ---")
for i in range(len(chunks) - 1):
    graph[i].append(i + 1)
    # Optional: If you want a two-way connection (undirected graph)
    # graph[i+1].append(i)

print("Sequential edges added.")


# --- 3. Add Similarity Edges ---
#  Connect chunks that are semantically similar.
print("\n---  Calculating similarities and adding similarity edges... ---")

# First, calculate the cosine similarity matrix for all embeddings
similarity_matrix = cosine_similarity(embeddings)

# Define a threshold for what we consider "similar"
# This is a key parameter to tune. A higher threshold means fewer, more relevant links.
SIMILARITY_THRESHOLD = 0.80

# Iterate through the matrix to find pairs of chunks above the threshold
# We use np.where to find the indices efficiently
similar_pairs = np.where(similarity_matrix > SIMILARITY_THRESHOLD)

for i, j in zip(*similar_pairs):
    if i != j:  # Don't connect a chunk to itself
        # Add an edge between the two similar chunks
        if j not in graph[i]:
            graph[i].append(j)
        # Optional: For an undirected graph
        # if i not in graph[j]:
        #     graph[j].append(i)

print(f"Similarity edges added with threshold > {SIMILARITY_THRESHOLD}.")


# --- 4. Verification (Optional) ---
# Let's inspect the connections for a specific chunk (e.g., the first one).
chunk_id_to_inspect = 0
connected_nodes = graph[chunk_id_to_inspect]

print(f"\n---  Graph built successfully. ---")
print(f"\n---  Inspecting connections for Chunk {chunk_id_to_inspect}: ---")
print("Original Chunk Text:")
print(chunks[chunk_id_to_inspect])
print("\nIt is connected to the following chunks:")
for node_id in connected_nodes:
    connection_type = "Sequential" if node_id == chunk_id_to_inspect + 1 else "Similarity"
    print(f"  - Chunk {node_id} ({connection_type})")
    # print(f"    Text: {chunks[node_id][:100]}...") # Uncomment to see text snippet

5th Step-User query and initial retrieval of the query based on vector databse only ,we will embed the query and then ask the database and ask ChromaDb to return top 3 most similar results to our query

In [ ]:
def query_vector_db_with_scores(query: str, collection, model, n_results=3):
    """
    Takes a user query, embeds it, and retrieves the most similar chunks
    and their scores from the ChromaDB collection.
    """
    # 1. Embed the user query
    query_embedding = model.encode(query).tolist()

    # 2. Query the collection
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    # 3. Extract the document chunks and their distances
    retrieved_chunks = results['documents'][0]
    distances = results['distances'][0]

    return retrieved_chunks, distances

# --- Let's Test the Function ---

query = "How does AuraFlow handle security and ethics?"
print(f"---  Query: {query} ---")

# Get both the chunks and their scores
initial_chunks, distances = query_vector_db_with_scores(query, collection, model)

# Display the results with scores
print("\n---  Top 3 initial results from Vector DB: ---")
for i, (chunk, dist) in enumerate(zip(initial_chunks, distances)):
    # Calculate a more intuitive confidence score (1.0 is a perfect match)
    confidence = 1 - dist

    print(f"\nResult {i+1} (Confidence: {confidence:.2%}, Distance: {dist:.4f}):")
    print(chunk)

6)From the retrieved top k chunks we will find the neighbours of the graph structure created (basically)explore the graph structue

In [ ]:
def expand_with_graph(initial_chunks, graph, all_chunks):
    """
    Expands the initial list of chunks by traversing the graph to find neighbors.
    """
    # Use a set to automatically handle duplicates
    expanded_chunks_set = set(initial_chunks)

    # Create a quick lookup map from chunk text to its index
    chunk_to_id = {chunk: i for i, chunk in enumerate(all_chunks)}

    # Iterate through the initial chunks to find their neighbors
    for chunk_text in initial_chunks:
        # 1. Find the node ID (index) of the current chunk
        node_id = chunk_to_id.get(chunk_text)
        if node_id is None:
            continue # Should not happen if all_chunks is correct

        # 2. Get all neighbors of this node from the graph
        neighbor_ids = graph.get(node_id, [])

        # 3. Add the text of each neighbor to our set
        for neighbor_id in neighbor_ids:
            neighbor_text = all_chunks[neighbor_id]
            expanded_chunks_set.add(neighbor_text)

    return list(expanded_chunks_set)

# --- Let's Test the Function ---

# We'll use the 'initial_chunks' retrieved from the last step.
# If you don't have it, run this line to generate it for a query:
# initial_chunks, _ = query_vector_db_with_scores("What is the Cognition Core?", collection, model)

print(f"---  Number of initial chunks: {len(initial_chunks)} ---")
print("Initial Chunks:", initial_chunks)

# Run the expansion
graph_expanded_chunks = expand_with_graph(initial_chunks, graph, chunks)

print(f"\n---  Number of chunks after graph expansion: {len(graph_expanded_chunks)} ---")
print("\n---  Expanded Chunks (Initial + Neighbors): ---")
for i, chunk in enumerate(graph_expanded_chunks):
    print(f"\nChunk {i+1}:")
    print(chunk)

7)Combining the Extracted information from both of them and using it and finding top queiries suitable for generation (which are to be given to LLM) , Here we will take the combined initial +graph queries and calculate the direct similarity of each one of them with the  original query


In this we have created a re rank function to rank the similarity with the initial queries

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def rerank_chunks(query: str, chunks_to_rank: list, model, top_n=5):
    """
    Reranks a list of chunks based on their semantic similarity to a query.

    Args:
        query (str): The user's original query.
        chunks_to_rank (list): The combined list of chunks to be reranked.
        model: The SentenceTransformer embedding model.
        top_n (int): The number of top chunks to return.

    Returns:
        list: A sorted list of the top_n most relevant chunk texts.
    """
    # 1. Embed the query and the chunks
    query_embedding = model.encode(query)
    chunk_embeddings = model.encode(chunks_to_rank)

    # 2. Calculate cosine similarity between the query and all chunks
    # Reshape query embedding to be a 2D array for the function
    similarities = cosine_similarity([query_embedding], chunk_embeddings)[0]

    # 3. Pair each chunk with its similarity score
    scored_chunks = list(zip(chunks_to_rank, similarities))

    # 4. Sort the chunks by score in descending order
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    # 5. Return the text of the top_n chunks
    top_chunks = [chunk for chunk, score in scored_chunks[:top_n]]

    return top_chunks

# --- Let's Test the Function ---

# Assume 'graph_expanded_chunks' from the previous step is available.
# Let's also define the query we used to get these chunks.
query = "What is the Cognition Core and how does it work?"

# For demonstration, let's create 'graph_expanded_chunks' if it's not in memory
if 'graph_expanded_chunks' not in locals():
    initial_chunks, _ = query_vector_db_with_scores(query, collection, model, n_results=3)
    graph_expanded_chunks = expand_with_graph(initial_chunks, graph, chunks)

print(f"---  Reranking {len(graph_expanded_chunks)} combined chunks... ---")

# Run the reranking
reranked_chunks = rerank_chunks(query, graph_expanded_chunks, model, top_n=5)

# Display the final, most relevant chunks
print(f"\n---  Top {len(reranked_chunks)} Reranked & Filtered Chunks: ---")
for i, chunk in enumerate(reranked_chunks):
    print(f"\nRank {i+1}:")
    print(chunk)

8)Concatenate the top ranked chunks into the context window which is to be given to LLM for generation

In [ ]:
def merge_chunks(chunks: list) -> str:
    """
    Concatenates a list of text chunks into a single string,
    separated by double newlines.

    Args:
        chunks (list): A list of strings (the reranked chunks).

    Returns:
        str: A single string containing the merged context.
    """
    return "\n\n---\n\n".join(chunks)

if 'reranked_chunks' not in locals():
    # If you ran the previous step, this list will be populated with actual data.
    reranked_chunks = [
        "The Cognition Core is the central reasoning engine of an AuraFlow instance. Unlike traditional neural networks, it utilizes a proprietary Probabilistic Logic Network (PLN) for decision-making.",
        "The Cognition Core is responsible for high-level task decomposition, planning, and final response synthesis. Its performance is heavily dependent on the quality of processed data it receives from the Data Weavers.",
        "The Cognition Core uses its PLN to analyze the data, reason about the query, and formulate a plan or response."
    ]

# Merge the reranked chunks into the final context window
final_context = merge_chunks(reranked_chunks)

# --- Verification ---
print("---  Final Merged Context Ready for LLM ---")
print(final_context)

9)Using LLM for final query output generation-
 I am using Gemini for generation

API KEY SETUP

In [ ]:
# Install the library for Google's Generative AI
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata
import textwrap

# Configure the API key from Colab secrets
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print(" API Key configured successfully!")
except userdata.SecretNotFoundError:
    print(' ERROR: Secret "GOOGLE_API_KEY" not found. Please follow Step 2 to set it up.')
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Final generation using LLM


In [ ]:
import google.generativeai as genai
import textwrap

# This code assumes your API key is already configured.

def generate_llm_answer(query: str, context: str) -> str:
    """
    Generates a final answer using the Gemini LLM based on the query and context.
    """
    prompt_template = f"""
    Answer the following query based only on the provided context.
    If the context does not contain the answer, state that the information is not available in the document.
    Be concise and do not add any information that is not present in the context.

    CONTEXT:
    {context}

    QUERY:
    {query}

    ANSWER:
    """

    try:

        model = genai.GenerativeModel('gemini-2.5-flash')

        response = model.generate_content(prompt_template)

        return response.text
    except Exception as e:
        return f"An error occurred while generating the answer: {e}"

# --- EXAMPLE EXECUTION ---
query = "What is the Cognition Core and how does it work?"

if 'final_context' not in locals():
    final_context = "The Cognition Core is the central reasoning engine of an AuraFlow instance. It utilizes a Probabilistic Logic Network (PLN) for decision-making, allowing it to handle uncertainty. It is responsible for task decomposition, planning, and response synthesis."

answer = generate_llm_answer(query, final_context)

# ---  FINAL, FORMATTED OUTPUT ---

# 1. Print the user query
print("---  USER QUERY ---")
print(query)
print("\n" + "="*50 + "\n")

# 2. Print the context that was sent to the LLM (with text wrap)
print("---  CONTEXT PROVIDED TO LLM ---")
print(textwrap.fill(final_context, width=80))
print("\n" + "="*50 + "\n")

# 3. Print the final answer from the LLM (with text wrap)
print("---  FINAL LLM-GENERATED ANSWER ---")
print(textwrap.fill(answer, width=80))